In [12]:
import os
from tqdm import tqdm
import cv2
import pandas as pd
%matplotlib inline

# Read from dataset

In [2]:
df = pd.read_excel('../ticket table by categories.xlsx',sheet_name='Sheet1',header=0)
print(len(df))
df.head()

200


,Ticket,Category,Name,Recording method,Note,Not whole screen,Poor quality,Shaky,Voice-assistant related,Other,Frame
0,NGC3D-258315,screen slide,FM,Camera,Not whole screen,1.0,NaN,NaN,NaN,NaN,NaN
1,CCS2BUG-586,frame error,AA connected phone receive message,Capture/Emulator,NaN,NaN,NaN,NaN,NaN,NaN,"103,288,499"
2,NCG3D-263526,list - slider,Half Turn List (2),Camera,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NCG3D-226483,screen slide,swipe left,Camera,Dark screen,NaN,NaN,NaN,NaN,1.0,NaN
4,NCG3D-281808,chaotic switch,connect iphone,Camera,2 bugs,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
for i in range(len(df)):
    if pd.isna(df.loc[i,'Frame']) or df.loc[i,'Category'] != 'frame error' or df.loc[i,'Recording method'] != 'Capture/Emulator':
        df.drop(labels=i,axis=0,inplace=True)

print(len(df))
df = df.reset_index(drop = True)
df = df[['Name','Frame']]
df.head()

51


,Name,Frame
0,AA connected phone receive message,"103,288,499"
1,AM-FM CP AM-FM,"110,177,228,279,327,374,414,457,499,549,600,64..."
2,black screen when trigger Siri,"548,854"
3,clock issue,"27,84,206,248,573,716"
4,connect CP then connect AA,"380,538,543,746,753"


# Write videos

In [4]:
def split_csv(str):
    answer = []
    for number in str.split(','):
        answer.append(int(number))
    return answer

In [5]:
def get_frame_number(filename):
    return int(filename.split('_')[0][5:])

In [18]:
FRAME_PATH = '../frames/frame error/'

for root, dirs, files in tqdm(os.walk(FRAME_PATH)):
    video_name = root.split('/')[-1]
    if len(files) > 0 and video_name in df['Name'].values:
        print(video_name, len(files))
        try:
            if not os.path.exists(f'./data/{video_name}'):
                os.makedirs(f'./data/{video_name}')
        except OSError:
            print ('Error: Creating directory of data')
            break
        
        height, width, _ = cv2.imread(os.path.join(root, files[0])).shape   
        shape = (width, height)
        flicker_frames = split_csv(df.loc[df['Name'] == video_name]['Frame'].values[0])
        current = 0
        files = sorted(files,key=get_frame_number)

        start = 0
        end = 0
        eof = len(files)
        _stride = 5
        while end < eof:
            end = start
            if end + 16 <= eof:
                end += 16
            else:
                padding = end + 16 - eof
                end = eof
                start -= padding

            target = 0
            for flicker in flicker_frames:
                if flicker >= start and flicker < end:
                    target = 1
                    break
            
            out = cv2.VideoWriter(f'./data/{video_name}/{video_name}_{start}_{end-1}_{target}.mp4',cv2.VideoWriter_fourcc(*'mp4v'), 16, shape)
            for i in range(start, end):
                frame = cv2.imread(os.path.join(root,files[i]))
                out.write(frame)
        
            start += _stride

0it [00:00, ?it/s]

black screen when trigger Siri 1380


2it [00:57, 28.99s/it]

long press 2 widgets on 2nd page switch to 1st page 168


5it [01:07, 11.34s/it]

...

116it [22:28, 13.04s/it]

press phone, white screen 383


117it [22:48, 14.61s/it]

CP map CP 439


120it [23:04, 11.54s/it]
